In [1]:
import json
from ckiptagger import construct_dictionary, WS, POS
ws = WS("./data")

C:\Users\Tibame_T14\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(


In [2]:
with open('./youtube/醫療保健text.json') as f:
    data = json.loads(f.read())
with open('./stopword.txt', encoding='utf-8') as f:
    stop = f.read()

In [3]:
stopword = stop.split('\n')

In [4]:
# 使用分好詞的資料
alltext = []
for file in data.values():
    for name, texts in file.items():
        word_sentence_list = ws(list(texts))
        text_list = []
        for i in range(len(texts)):
            for word in word_sentence_list[i]:
                if word not in stopword:
                    text_list.append(word)
        if text_list != []:
            alltext.append(text_list)

In [6]:
from gensim.models import Word2Vec

In [42]:
model = Word2Vec(sentences=alltext, min_count=1, workers=4)

In [43]:
from gensim.similarities import WordEmbeddingSimilarityIndex

In [44]:
termsim_index = WordEmbeddingSimilarityIndex(model.wv)

In [45]:
from gensim.corpora import Dictionary

In [46]:
dictionary = Dictionary(alltext)

In [47]:
bow_corpus = [dictionary.doc2bow(document) for document in alltext]

In [48]:
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix

In [49]:
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix)

100%|███████████████████████████████████████████████████████████████████████████| 26687/26687 [00:50<00:00, 527.94it/s]


In [50]:
sims = docsim_index[dictionary.doc2bow(['瘦身','健康','減脂'])]
sims

array([0.9241805 , 0.8964102 , 1.        , 1.        , 1.        ,
       0.84100103, 0.9258647 , 0.92797804, 0.88928807, 0.88125074,
       0.91989326, 0.9914752 , 0.9784206 , 0.6837492 , 0.92366385,
       0.83623815, 0.95644116, 0.9360825 , 0.9879719 , 1.        ,
       1.        , 0.98929095, 0.8474557 , 0.8002254 , 0.77783054,
       1.        , 0.95105344, 0.90492594, 0.9988092 , 1.        ,
       0.8589978 , 0.8749839 , 0.9739026 , 0.8430411 , 0.952969  ,
       1.        , 1.        , 0.867091  , 0.98867935, 0.9014647 ,
       0.98143166, 0.833838  , 0.9139205 , 0.9005903 , 0.95796204,
       0.94906855, 0.76047873, 0.8324393 , 0.82797366, 0.80107266,
       0.92393607, 0.8427716 , 1.        , 0.9072678 , 0.96913815,
       0.8937018 , 0.90210193, 0.81601787, 0.8405052 , 0.94213027,
       0.88259524, 0.8299928 , 0.76692617, 0.87744164, 0.7344234 ,
       0.5995979 , 0.8409891 , 0.        , 0.96803945, 0.89157915,
       0.847083  , 1.        , 0.7660976 , 0.8356306 , 0.89344

In [64]:
model.wv.most_similar(['瘦身','健康','減脂'])

[('重點', 0.9997672438621521),
 ('不錯', 0.9997509717941284),
 ('介紹', 0.999735414981842),
 ('完全', 0.9997349381446838),
 ('平常', 0.9997339844703674),
 ('相信', 0.9997309446334839),
 ('辦法', 0.9997294545173645),
 ('控制', 0.9997289180755615),
 ('話', 0.9997275471687317),
 ('ok', 0.9997250437736511)]

In [66]:
model.wv.similar_by_word('瘦身')

[('穿', 0.9989466071128845),
 ('換', 0.9989323616027832),
 ('毛孔', 0.9989060163497925),
 ('防曬', 0.9989004731178284),
 ('重點', 0.9988945126533508),
 ('新', 0.998885989189148),
 ('美白', 0.998884379863739),
 ('12', 0.9988680481910706),
 ('心', 0.9988675713539124),
 ('製作', 0.9988542199134827)]

In [92]:
import numpy as np
doc = np.array([model.wv[alltext[i]].mean(axis=0) for i in range(len(alltext))])

In [96]:
import pandas as pd
data = pd.DataFrame(doc)

In [98]:
data.to_csv('w2vmeandata')

In [100]:
from gensim.models import TfidfModel

In [101]:
model2 = TfidfModel(bow_corpus)

In [132]:
data2 = []
for i in range(len(alltext)):
    s = np.array([0 for _ in range(100)],dtype='float32')
    d = dictionary.doc2idx(alltext[i])
    vector = dict(model2[bow_corpus[i]])
    to = 0
    for l in range(len(alltext[i])):
        s += model.wv[alltext[i][l]]*vector[d[l]]
        to += vector[d[l]]
    s /= to
    data2.append(s)

In [138]:
data2 = pd.DataFrame(data2)

In [140]:
data2.to_csv('w2vtfidfdata')